In [ ]:
#!/usr/bin/env python3
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib.pyplot as plt
# plt.style.use('../clint.mpl')
from matplotlib.colors import LogNorm

import boost_histogram as bh
import pickle as pl

from pygama import DataGroup
import pygama.io.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf

Load background data from the run of interest

In [ ]:
bkg_dg = DataGroup('cage.json', load=True)
bkg_run = 'run==68 and skip==False'
# bkg_run = 'run==34 and skip==False'
bkg_dg.fileDB.query(bkg_run, inplace=True)
view_cols = ['runtype', 'run', 'cycle', 'startTime', 'runtime', 'threshold']
print(bkg_dg.fileDB[view_cols])

Load alpha data from the run of interest

In [ ]:
alpha_dg = DataGroup('cage.json', load=True)
alpha_run = 'run==70 and skip==False'
# alpha_run = "run>50 and runtype=='alp'"
# alpha_run = 'run==50 and skip==False'
alpha_dg.fileDB.query(alpha_run, inplace=True)
view_cols = ['runtype', 'run', 'cycle', 'startTime', 'runtime', 'threshold']
print(alpha_dg.fileDB[view_cols])

Plot raw energy histogram of the background data, then alpha data, from DSP files
First get dsp data, runtimes so we can normalize by time

First get file list and load background energy data (numpy array)

In [ ]:
# get file list and load background energy data (numpy array)
lh5_dir = os.path.expandvars(bkg_dg.config['lh5_dir'])
bkg_dsp_list = lh5_dir + bkg_dg.fileDB['dsp_path'] + '/' + bkg_dg.fileDB['dsp_file']
bkg_edata = lh5.load_nda(bkg_dsp_list, ['trapEmax'], 'ORSIS3302DecoderForEnergy/dsp')
bkg_rt_min = bkg_dg.fileDB['runtime'].sum()
bkg_u_start = bkg_dg.fileDB.iloc[0]['startTime']
bkg_t_start = pd.to_datetime(bkg_u_start, unit='s') # str

Get file list for alpha data

In [ ]:
# get file list and load alpha energy data (numpy array)
lh5_dir = os.path.expandvars(alpha_dg.config['lh5_dir'])
alpha_dsp_list = lh5_dir + alpha_dg.fileDB['dsp_path'] + '/' + alpha_dg.fileDB['dsp_file']
alpha_edata = lh5.load_nda(alpha_dsp_list, ['trapEmax'], 'ORSIS3302DecoderForEnergy/dsp')
alpha_rt_min = alpha_dg.fileDB['runtime'].sum()
alpha_u_start = alpha_dg.fileDB.iloc[0]['startTime']
alpha_t_start = pd.to_datetime(alpha_u_start, unit='s') # str

Create background histogram

In [ ]:
# Create background histogram, normalized by runtime
# elo, ehi, epb = 0, 25000, 10 
elo, ehi, epb = 0, 40000, 10 
etype = 'trapEmax'

bkg_ene_uncal = bkg_edata[etype]
bkg_hist, bins, _ = pgh.get_hist(bkg_ene_uncal, range=(elo, ehi), dx=epb)

# normalize by runtime in seconds
bkg_hist_rt = np.divide(bkg_hist, bkg_rt_min * 60)

Create alpha histogram

In [ ]:
# Create alpha histogram, normalized by runtime
etype = 'trapEmax'
# elo, ehi, epb = 0, 25000, 10
elo, ehi, epb = 0, 40000, 10 

alpha_ene_uncal = alpha_edata[etype]
alpha_hist, bins, _ = pgh.get_hist(alpha_ene_uncal, range=(elo, ehi), dx=epb)

# normalize by runtime in seconds
alpha_hist_rt = np.divide(alpha_hist, alpha_rt_min * 60)

Plot background spectrum

In [ ]:
# Plot the histograms
plt.semilogy(bins[1:], bkg_hist_rt, ds='steps', c='b', lw=1,
            label=f'{etype}, {bkg_rt_min:.2f} mins')

plt.xlabel(etype, ha='right', x=1)
plt.ylabel('cts / sec', ha='right', y=1)
plt.legend()
plt.title(f'Background run: {bkg_run}')
plt.show()

Plot alpha spectrum

In [ ]:
# Plot the alpha histogram
plt.semilogy(bins[1:], alpha_hist_rt, ds='steps', c='b', lw=1,
            label=f'{etype}, {alpha_rt_min:.2f} mins')

plt.xlabel(etype, ha='right', x=1)
plt.ylabel('cts / sec', ha='right', y=1)
plt.legend()
plt.title(f'Alpha run: {alpha_run}')
plt.show()

Subtract the Background histogram from the alpha histogram

In [ ]:
hist_bkg_sub = alpha_hist_rt - bkg_hist_rt

Plot background-subtracted histogram 

In [ ]:
# Plot the background-subtracted histogram
plt.semilogy(bins[1:], hist_bkg_sub, ds='steps', c='b', lw=1,
            label=f'{etype}, background subtracted')

plt.xlabel(etype, ha='right', x=1)
plt.ylabel('cts / sec', ha='right', y=1)
plt.legend()
plt.title(f'Background subtracted, Alpha run: {alpha_run}')
plt.show()

Load hit data from local directory

In [ ]:
lh5_dir = bkg_dg.lh5_user_dir #if user else dg.lh5_dir
print(lh5_dir)
# exit()
bkg_hit_list = lh5_dir + bkg_dg.fileDB['hit_path'] + '/' + bkg_dg.fileDB['hit_file']
bkg_df_hit = lh5.load_dfs(bkg_hit_list, ['trapEmax', 'bl','bl_sig','A_10','ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max'], 'ORSIS3302DecoderForEnergy/hit')
# bkg_df_hit = lh5.load_dfs(bkg_hit_list, ['trapEmax', 'bl','bl_sig','A_10','ts_sec', 'dcr'], 'ORSIS3302DecoderForEnergy/hit')
# print(df_hit)
print(bkg_df_hit.columns)

bkg_dsc = bkg_df_hit[['bl','bl_sig','A_10','ts_sec', 'dcr_raw']].describe()
# bkg_dsc = bkg_df_hit[['bl','bl_sig','A_10','ts_sec', 'dcr']].describe()
print(bkg_dsc)

In [ ]:
lh5_dir = alpha_dg.lh5_user_dir #if user else dg.lh5_dir
print(lh5_dir)
alpha_hit_list = lh5_dir + alpha_dg.fileDB['hit_path'] + '/' + alpha_dg.fileDB['hit_file']
print(alpha_hit_list)
alpha_df_hit = lh5.load_dfs(alpha_hit_list, ['trapEmax', 'bl','bl_sig','A_10','ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max'], 'ORSIS3302DecoderForEnergy/hit')
# alpha_df_hit = lh5.load_dfs(alpha_hit_list, ['trapEmax', 'bl','bl_sig','A_10','ts_sec', 'dcr_raw'], 'ORSIS3302DecoderForEnergy/hit')
# print(df_hit)
print(alpha_df_hit.columns)

alpha_dsc = alpha_df_hit[['bl','bl_sig','A_10','ts_sec', 'dcr_raw', 'dcr_ftp', 'dcr_max']].describe()
# alpha_dsc = alpha_df_hit[['bl','bl_sig','A_10','ts_sec', 'dcr_raw']].describe()
print(alpha_dsc)

Now plot baseline vs E

In [ ]:
# Background Baseline vs E
elo, ehi, epb = 0, 50, 1
blo, bhi, bpb = 0, 10000, 100
nbx = int((ehi-elo)/epb)
nby = int((bhi-blo)/bpb)

h = plt.hist2d(bkg_df_hit['trapEmax'], bkg_df_hit['bl'], bins=[nbx,nby],
                       range=[[elo, ehi], [blo, bhi]], cmap='jet')

cb = plt.colorbar(h[3], ax=plt.gca())
plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('bl', ha='right', y=1)
plt.title('Baseline, background')
# plt.tight_layout()
plt.show()

In [ ]:
# Alpha Baseline vs E
elo, ehi, epb = 0, 50, 1
blo, bhi, bpb = 0, 10000, 100
nbx = int((ehi-elo)/epb)
nby = int((bhi-blo)/bpb)

h = plt.hist2d(alpha_df_hit['trapEmax'], alpha_df_hit['bl'], bins=[nbx,nby],
                       range=[[elo, ehi], [blo, bhi]], cmap='jet')

cb = plt.colorbar(h[3], ax=plt.gca())
plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('bl', ha='right', y=1)
plt.title('Baseline, alpha')
# plt.tight_layout()
plt.show()

Make a cut based on BL value

In [ ]:
# make a formal baseline cut from 1d histogram
hE, bins, vE = pgh.get_hist(alpha_df_hit['bl'], range=(blo, bhi), dx=bpb)
xE = bins[1:]
plt.semilogy(xE, hE, c='b', ds='steps')

bl_cut_lo, bl_cut_hi = 8000,10000
plt.axvline(bl_cut_lo, c='r', lw=1)
plt.axvline(bl_cut_hi, c='r', lw=1)

plt.xlabel('bl', ha='right', x=1)
plt.ylabel('counts', ha='right', y=1)
plt.show()

Use BL cut from above, create A/E

In [ ]:
# Background A_10/trapEmax vs trapEmax (A/E vs E)

# use baseline cut
bkg_df_cut = bkg_df_hit.query('bl > 8000 and bl < 10000').copy()

# add new A/E column
bkg_df_cut['aoe'] = bkg_df_cut['A_10'] / bkg_df_cut['trapEmax']

In [ ]:
elo, ehi, epb = 0, 35000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)

energy_hist, bins = np.histogram(bkg_df_cut['trapEmax'], bins=nbx,
                range=[elo, ehi])

plt.semilogy(bins[1:], energy_hist, ds='steps', c='b', lw=1)
# = plt.hist(bkg_df_cut['trapEmax'], bins=nbx,
#                 range=[elo, ehi], cmap='jet', norm=LogNorm())

plt.show()

In [ ]:
# Alpha A_10/trapEmax vs trapEmax (A/E vs E)

# use baseline cut
alpha_df_cut = alpha_df_hit.query('bl > 8000 and bl < 10000').copy()

# add new A/E column
alpha_df_cut['aoe'] = alpha_df_cut['A_10'] / alpha_df_cut['trapEmax']

In [ ]:
elo, ehi, epb = 0, 35000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)

alpha_energy_hist, bins = np.histogram(alpha_df_cut['trapEmax'], bins=nbx,
                range=[elo, ehi])

plt.semilogy(bins[1:], energy_hist, ds='steps', c='b', lw=1)
# = plt.hist(bkg_df_cut['trapEmax'], bins=nbx,
#                 range=[elo, ehi], cmap='jet', norm=LogNorm())

plt.show()

Plot A/E vs E

In [ ]:
# Background A/E
alo, ahi, apb = 0.0, 0.09, 0.0001
elo, ehi, epb = 0, 25000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)
nby = int((ahi-alo)/apb)

h = plt.hist2d(bkg_df_cut['trapEmax'], bkg_df_cut['aoe'], bins=[nbx,nby],
                range=[[elo, ehi], [alo, ahi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('A/E', ha='right', y=1)
plt.title('Background A/E')
plt.tight_layout()
plt.show()

In [ ]:
# Alpha A/E
alo, ahi, apb = 0.0, 0.09, 0.0001
elo, ehi, epb = 0, 25000, 10
# elo, ehi, epb = 0, 3000, 10
# elo, ehi, epb = 0, 6000, 10
        

nbx = int((ehi-elo)/epb)
nby = int((ahi-alo)/apb)

h = plt.hist2d(alpha_df_cut['trapEmax'], alpha_df_cut['aoe'], bins=[nbx,nby],
                range=[[elo, ehi], [alo, ahi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('A/E', ha='right', y=1)
plt.title('Alpha A/E')
plt.tight_layout()
plt.show()

Plot DCR vs E

In [ ]:
# Bkg show DCR vs E
etype = 'trapEmax'
#       elo, ehi, epb = 0, 6000, 10
elo, ehi, epb = 0, 25000, 10
# dlo, dhi, dpb = -1000, 1000, 10
# dlo, dhi, dpb = -0.25, 0.25, 0.05
# dlo, dhi, dpb = -100., 100., 1
dlo, dhi, dpb = -1000., 0., 1

nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

h = plt.hist2d(bkg_df_cut['trapEmax'], bkg_df_cut['dcr_raw'], bins=[nbx,nby],
                range=[[elo, ehi], [dlo, dhi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('DCR_raw', ha='right', y=1)
plt.title('Background DCR')
# plt.tight_layout()
# plt.savefig('./plots/bkg_dcr_raw_run34.png', dpi=200)
plt.show()

In [ ]:
# Alpha show DCR vs E
etype = 'trapEmax'
#       elo, ehi, epb = 0, 6000, 10
elo, ehi, epb = 0, 25000, 10
# dlo, dhi, dpb = -1000, 1000, 10
# dlo, dhi, dpb = -0.25, 0.25, 0.0005
# dlo, dhi, dpb = -100., 100., 1
# dlo, dhi, dpb = -1000., 0., 1
dlo, dhi, dpb = -1000., 0., 1

nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

h = plt.hist2d(alpha_df_cut['trapEmax'], alpha_df_cut['dcr_raw'], bins=[nbx,nby],
                range=[[elo, ehi], [dlo, dhi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('DCR_raw', ha='right', y=1)
plt.title('Alpha DCR')
# plt.tight_layout()
# plt.savefig('./plots/alpha_dcr_raw_run35.png', dpi=200)
plt.show()

In [ ]:
#create dcr/E for background
bkg_df_cut['dcr_norm'] = bkg_df_cut['dcr_raw'] / bkg_df_cut['trapEmax']

#create dcr/E for alpha
alpha_df_cut['dcr_norm'] = alpha_df_cut['dcr_raw'] / alpha_df_cut['trapEmax']

In [ ]:
# Bkg show DCR/E vs E
etype = 'trapEmax'
#       elo, ehi, epb = 0, 6000, 10
elo, ehi, epb = 0, 25000, 10
# dlo, dhi, dpb = -1000, 1000, 10
# dlo, dhi, dpb = -0.25, 0.25, 0.05
# dlo, dhi, dpb = -100., 100., 1
# dlo, dhi, dpb = -1000., 0., 1
# dlo, dhi, dpb = -0.5, 0.025, 0.0005
dlo, dhi, dpb = -0.075, 0.025, 0.0005

nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

bkg_dcr_hist = plt.hist2d(bkg_df_cut['trapEmax'], bkg_df_cut['dcr_norm'], bins=[nbx,nby],
                range=[[elo, ehi], [dlo, dhi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('DCR_norm', ha='right', y=1)
plt.title('Background DCR')
plt.tight_layout()
# plt.savefig('./plots/bkg_dcr_norm_run68.pdf', dpi=200)
plt.show()

In [ ]:
# Alpha show DCR vs E
etype = 'trapEmax'
#       elo, ehi, epb = 0, 6000, 10
elo, ehi, epb = 0, 25000, 10
# dlo, dhi, dpb = -1000, 1000, 10
# dlo, dhi, dpb = -0.25, 0.25, 0.0005
# dlo, dhi, dpb = -100., 100., 1
# dlo, dhi, dpb = -1000., 0., 1
# dlo, dhi, dpb = -1000., 0., 1
# dlo, dhi, dpb = -0.08, 0.025, 0.0005
dlo, dhi, dpb = -0.1, 0.025, 0.0005

nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

alpha_dcr_hist = plt.hist2d(alpha_df_cut['trapEmax'], alpha_df_cut['dcr_norm'], bins=[nbx,nby],
                range=[[elo, ehi], [dlo, dhi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('DCR_norm', ha='right', y=1)
plt.title('Alpha DCR')
plt.tight_layout()
# plt.savefig('./plots/alpha_dcr_norm_run70.pdf', dpi=200)
plt.show()

In [ ]:
Alpha_dcr_cut = alpha_df_cut.query('trapEmax > 5000 and trapEmax < 15000 and dcr_norm > -0.0515 and dcr_norm < -0.02').copy()

In [ ]:
energy = Alpha_dcr_cut['trapEmax']
print(len(energy))

In [ ]:
# Alpha show DCR vs E
etype = 'trapEmax'
#       elo, ehi, epb = 0, 6000, 10
elo, ehi, epb = 0, 25000, 10
# dlo, dhi, dpb = -1000, 1000, 10
# dlo, dhi, dpb = -0.25, 0.25, 0.0005
# dlo, dhi, dpb = -100., 100., 1
# dlo, dhi, dpb = -1000., 0., 1
# dlo, dhi, dpb = -1000., 0., 1
dlo, dhi, dpb = -0.06, 0.025, 0.0005

nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

hist = bh.numpy.histogram2d(Alpha_dcr_cut['trapEmax'], Alpha_dcr_cut['dcr_norm'], bins=[nbx,nby],
                range=[[elo, ehi], [dlo, dhi]], histogram=bh.Histogram)

with open('test_save_hist.pkl', 'wb') as f:
    pl.dump(hist, f)



# alphas = plt.hist2d(Alpha_dcr_cut['trapEmax'], Alpha_dcr_cut['dcr_norm'], bins=[nbx,nby],
#                 range=[[elo, ehi], [dlo, dhi]], cmap='jet', norm=LogNorm())

# plt.xlabel('trapEmax', ha='right', x=1)
# plt.ylabel('DCR_norm', ha='right', y=1)
# plt.title(f"{len(Alpha_dcr_cut['trapEmax'])} Alphas after cut")
# plt.tight_layout()
# plt.savefig('./plots/cut_alpha_dcr_norm_run35.png', dpi=200)
# plt.show()

In [ ]:
elo, ehi, epb = 0, 25000, 10
dlo, dhi, dpb = -0.08, 0.025, 0.0005

nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

alpha_dcr_hist = bh.numpy.histogram2d(alpha_df_cut['trapEmax'], alpha_df_cut['dcr_norm'], bins=[nbx,nby],
                range=[[elo, ehi], [dlo, dhi]], histogram=bh.Histogram)

In [ ]:
with open('test2_save_hist.pkl', 'wb') as f:
    pl.dump(alpha_dcr_hist, f)

In [ ]:
with open('test_save_hist.pkl', 'rb') as f:
    h2 = pl.load(f)
    
print(h2)

In [ ]:
def plothist2d(h):
    return plt.pcolormesh(*h.axes.edges.T, h.view().T, cmap='plasma', norm=LogNorm())

In [ ]:
plothist2d(h2)

In [ ]:
with open('test2_save_hist.pkl', 'rb') as f:
    h3 = pl.load(f)
    
print(h3)

In [ ]:
plothist2d(h3)

In [ ]:
#3d hist with E, A/E, and DCR
#note: this is not useful to save because the file size is huge! ~4Gb!

# A/E

elo, ehi, epb = 0, 25000, 10
alo, ahi, apb = 0.0, 0.09, 0.0001
dlo, dhi, dpb = -0.1, 0.025, 0.0005

        

nbx = int((ehi-elo)/epb)
nby = int((ahi-alo)/apb)
nbz = int((dhi-dlo)/dpb)

energy_aoe_dcr_hist = bh.numpy.histogramdd((alpha_df_cut['trapEmax'], alpha_df_cut['aoe'], alpha_df_cut['dcr_norm']), bins=[nbx,nby, nbz],
                range=[[elo, ehi], [alo,ahi], [dlo, dhi]], histogram=bh.Histogram)

In [ ]:
with open('test3d_save_hist.pkl', 'wb') as f:
    pl.dump(energy_aoe_dcr_hist, f)

In [ ]:
# show Background DCR vs A/E
alo, ahi, apb = 0.0, 1, 0.0009
#         dlo, dhi, dpb = -1000, 1000, 10
#         dlo, dhi, dpb = -0.25, 0.25, 0.0005
# dlo, dhi, dpb = -1., 1, 0.0005
# dlo, dhi, dpb = -1000., 0.8, 1
dlo, dhi, dpb = -1000., 100., 1

nbx = int((ahi-alo)/apb)
print(nbx)
nby = int((dhi-dlo)/dpb)

h = plt.hist2d(bkg_df_cut['aoe'], bkg_df_cut['dcr_raw'], bins=[nbx,nby],
                range=[[alo, ahi], [dlo, dhi]], cmap='jet', norm=LogNorm())

plt.xlabel('A/E', ha='right', x=1)
plt.ylabel('DCR', ha='right', y=1)
# plt.savefig('./plots/zoom_alpha_dcr_norm_run31.png', dpi=200)
# plt.tight_layout()
plt.show()

In [ ]:
# show Background DCR vs A/E
# alo, ahi, apb = 0.0, 0.09, 0.0009
# #         dlo, dhi, dpb = -1000, 1000, 10
# #         dlo, dhi, dpb = -0.25, 0.25, 0.0005
# dlo, dhi, dpb = -0.085, 0.025, 0.0005
# # dlo, dhi, dpb = -1000., 0.8, 1

alo, ahi, apb = 0.0, 1, 0.0009
#         dlo, dhi, dpb = -1000, 1000, 10
#         dlo, dhi, dpb = -0.25, 0.25, 0.0005
# dlo, dhi, dpb = -1., 1, 0.0005
dlo, dhi, dpb = -1000., 100., 1

nbx = int((ahi-alo)/apb)
print(nbx)
nby = int((dhi-dlo)/dpb)

h = plt.hist2d(alpha_df_cut['aoe'], alpha_df_cut['dcr_raw'], bins=[nbx,nby],
                range=[[alo, ahi], [dlo, dhi]], cmap='jet', norm=LogNorm())

plt.xlabel('A/E', ha='right', x=1)
plt.ylabel('DCR', ha='right', y=1)
# plt.savefig('./plots/zoom_alpha_dcr_norm_run31.png', dpi=200)
# plt.tight_layout()
plt.show()

In [ ]:
import boost_histogram as bh

In [ ]:
# Alpha show DCR vs E
etype = 'trapEmax'
#       elo, ehi, epb = 0, 6000, 10
elo, ehi, epb = 0, 25000, 10
# dlo, dhi, dpb = -1000, 1000, 10
# dlo, dhi, dpb = -0.25, 0.25, 0.0005
# dlo, dhi, dpb = -100., 100., 1
# dlo, dhi, dpb = -1000., 0., 1
# dlo, dhi, dpb = -1000., 0., 1
dlo, dhi, dpb = -0.08, 0.025, 0.0005

nbx = int((ehi-elo)/epb)
nby = int((dhi-dlo)/dpb)

alpha_dcr_hist = plt.hist2d(alpha_df_cut['trapEmax'], alpha_df_cut['dcr_norm'], bins=[nbx,nby],
                range=[[elo, ehi], [dlo, dhi]], cmap='jet', norm=LogNorm())

plt.xlabel('trapEmax', ha='right', x=1)
plt.ylabel('DCR_norm', ha='right', y=1)
plt.title('Alpha DCR')
plt.tight_layout()
plt.savefig('./plots/alpha_dcr_norm_run70.pdf', dpi=200)
plt.show()